In [ ]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import csv


# consumer key, consumer secret, access token, access secret.
ckey = '***************'
csecret = '***************'
atoken = '***************'
asecret = '***************'


class listener(StreamListener):
    def on_data(self, data):
        try:
            all_data = json.loads(data) 
            time = all_data['created_at']
            user_id = all_data['user']['id']
            user_name = all_data['user']['name']
            user_followers = all_data['user']['followers_count']
            

            tweet = all_data['text']
            extended = 'not extended'
            if (tweet[:2] == 'RT'):
                rt = tweet.split(':', 1)[0]
                # RT && extended
                try:
                    tweet = all_data['retweeted_status']['extended_tweet']['full_text']
                    extended = 'RT extended'
                # RT && not extended
                except:
                    tweet = tweet.split(':', 1)[1]
            else:
                rt = ''
                # not RT && extended
                try:
                    tweet = all_data['extended_tweet']['full_text']
                    extended = 'extended'
                # not RT && not extended
                except:
                    tweet = tweet

            tweet = tweet.replace('\n',' ').replace('\r',' ').strip()
            
            print('-------------------------')
            print(time[:-11] + ' | ' + rt + ' | ' + str(extended))
            print(tweet)
            print('-------------------------')

            row = [time, rt, tweet, str(user_id), user_name, str(user_followers)]
            output = open('output.csv', 'a')
            wr = csv.writer(output, dialect='excel')
            wr.writerow(row)
            output.close()
        except:
            pass
        return True

    
    def on_error(self, status):
        print(status)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)
keywords = ['Trump', 'Biden']

twitterStream = Stream(auth, listener(), tweet_mode='extended')

twitterStream.filter(track=keywords, languages=['en'])